In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from public_tests import *
import utils_practice as utils
from utils import generate_split_viz, generate_tree_viz

%matplotlib inline

In [ ]:
X_train = np.array(
    [
        [1, 1, 1],
        [1, 0, 1],
        [1, 0, 0],
        [1, 0, 0],
        [1, 1, 1],
        [0, 1, 1],
        [0, 0, 0],
        [1, 0, 1],
        [0, 1, 0],
        [1, 0, 0],
    ]
)
y_train = np.array([1, 1, 0, 0, 1, 0, 0, 1, 1, 0])

In [ ]:
print(f"First few elements of X_train:\n {X_train[:5]}")
print(f"Type of X_train: {type(X_train)}")

In [ ]:
print(f"First few elements of y_train: {y_train[:5]}")
print(f"Type of y_train: {type(y_train)}")

In [ ]:
print("The shape of X_train is:", X_train.shape)
print("The shape of y_train is: ", y_train.shape)
print("Number of training examples (m):", len(X_train))

In [ ]:
# UNQ_C1
# GRADED FUNCTION: compute_entropy


def compute_entropy(y):
    """
    Computes the entropy for

    Args:
       y (ndarray): Numpy array indicating whether each example at a node is
           edible (`1`) or poisonous (`0`)

    Returns:
        entropy (float): Entropy at that node

    """
    # You need to return the following variables correctly
    entropy = 0.0

    ### START CODE HERE ###
    if len(y) == 0:
        return 0

    label_is_1 = y == 1
    p = np.mean(label_is_1)
    if p == 0 or p == 1:
        return 0
    entropy = -p * np.log2(p) - (1 - p) * np.log2(1 - p)

    return entropy

In [ ]:
# Compute entropy at the root node (i.e. with all examples)
# Since we have 5 edible and 5 non-edible mushrooms, the entropy should be 1"

print("Entropy at root node: ", compute_entropy(y_train))

# UNIT TESTS
compute_entropy_test(compute_entropy)
compute_entropy(np.array([0, 0]))

In [ ]:
# UNQ_C2
# GRADED FUNCTION: split_dataset


def split_dataset(X, node_indices, feature):
    """
    Splits the data at the given node into
    left and right branches

    Args:
        X (ndarray):             Data matrix of shape(n_samples, n_features)
        node_indices (list):     List containing the active indices. I.e, the samples being considered at this step.
        feature (int):           Index of feature to split on

    Returns:
        left_indices (list):     Indices with feature value == 1
        right_indices (list):    Indices with feature value == 0
    """

    # You need to return the following variables correctly
    left_indices = []
    right_indices = []

    ### START CODE HERE ###
    node_indices = np.asarray(node_indices)
    ones = X[node_indices, feature] == 1
    left_indices = node_indices[ones].tolist()
    right_indices = node_indices[~ones].tolist()
    ### END CODE HERE ###

    return left_indices, right_indices

In [ ]:
# Case 1

root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Feel free to play around with these variables
# The dataset only has three features, so this value can be 0 (Brown Cap), 1 (Tapering Stalk Shape) or 2 (Solitary)
feature = 0

left_indices, right_indices = split_dataset(X_train, root_indices, feature)

print("CASE 1:")
print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

# Visualize the split
generate_split_viz(root_indices, left_indices, right_indices, feature)

print()

# Case 2

root_indices_subset = [0, 2, 4, 6, 8]
left_indices, right_indices = split_dataset(X_train, root_indices_subset, feature)

print("CASE 2:")
print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

# Visualize the split
generate_split_viz(root_indices_subset, left_indices, right_indices, feature)

# UNIT TESTS
split_dataset_test(split_dataset)

In [ ]:
# UNQ_C3
# GRADED FUNCTION: compute_information_gain


def compute_information_gain(X, y, node_indices, feature):
    """
    Compute the information of splitting the node on a given feature

    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        feature (int):           Index of feature to split on

    Returns:
        cost (float):        Cost computed

    """
    # Split dataset
    left_indices, right_indices = split_dataset(X, node_indices, feature)

    # Some useful variables
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]

    # You need to return the following variables correctly
    information_gain = 0

    ### START CODE HERE ###

    h_node = compute_entropy(y_node)
    h_left, h_right = compute_entropy(y_left), compute_entropy(y_right)
    w_left, w_right = len(y_left) / len(y_node), len(y_right) / len(y_node)
    information_gain = h_node - (w_left * h_left + w_right * h_right)

    ### END CODE HERE ###

    return information_gain

In [ ]:
info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)

info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)

# UNIT TESTS
compute_information_gain_test(compute_information_gain)

In [ ]:
# UNQ_C4
# GRADED FUNCTION: get_best_split


def get_best_split(X, y, node_indices):
    """
    Returns the optimal feature and threshold value
    to split the node data

    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.

    Returns:
        best_feature (int):     The index of the best feature to split
    """

    # Some useful variables
    num_features = X.shape[1]

    # You need to return the following variables correctly
    best_feature = -1

    ### START CODE HERE ###

    most_info_gain = 0
    for feature_idx in range(X.shape[1]):
        info_gain = compute_information_gain(X, y, node_indices, feature_idx)
        if info_gain > most_info_gain:
            best_feature = feature_idx
            most_info_gain = info_gain

    ### END CODE HERE ##

    return best_feature

In [ ]:
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

# UNIT TESTS
get_best_split_test(get_best_split)

In [ ]:
tree = []


def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.

    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree.
        current_depth (int):    Current depth. Parameter used during recursive call.

    """

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        format_str = " " * current_depth + "-" * current_depth
        print(f"{format_str} {branch_name} leaf node with indices {node_indices}")
        return

    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node

    best_feature = get_best_split(X, y, node_indices)
    format_str = "-" * current_depth
    print(
        f"{format_str} Depth {current_depth}, {branch_name}: Split on feature: {best_feature}"
    )

    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))

    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth + 1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth + 1)

In [ ]:
build_tree_recursive(
    X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0
)
generate_tree_viz(root_indices, y_train, tree)